In [2]:
#limpio la memoria
rm(list=ls())
gc()

#cargo librerias
library("data.table")
library("ROCR")
library("rpart")
library("rpart.plot")
library("caret")

,used,(Mb),gc trigger,(Mb),limit (Mb),max used,(Mb)
Ncells,2253643,120.4,4424110,236.3,NA,3420747,182.7
Vcells,3804761,29.1,8388608,64.0,16384,5634034,43.0


In [3]:
setwd("~/cloudL/UA/labo2022/")

#Parametros entrada

#kcarpeta_datasets    <- "../input/laboratorio-de-implementacion-i-2021/"   #KAGGLE
kcarpeta_datasets    <- "./datasets/"                          #VM o Ubuntu

#Archivo con datos etiquetados para entrenamiento
karchivo_entrada      <-  paste0(kcarpeta_datasets, "competencia1_2022.csv")

#Formato para submit en Kaggle
#karchivo_score      <-  "../input/uamds2020ldi1f1/Sample_201910_Fase_I.txt"

#Separador de campos en archivos
kcampos_separador     <-  "\t"

#Campo que identifica las muestras
kcampo_id             <-  "numero_de_cliente"

#Campo que contiene la clase a estimar
kclase_nomcampo       <-  "clase_ternaria"

#Valor de interés
kclase_valor_positivo <-  "BAJA+2"

#Campos a borrar para el entrenamiento
kcampos_a_borrar      <-  c(kcampo_id,kclase_nomcampo,"foto_mes")

#Campo que contendrá a la variable objetivo generada
kobjetivo             <-  "clase"

#Identificación del modelo
kmodelo               <-  "02-RPART"

#Ganancia por TP
kTPGain               <-  78000

#Pérdida por FP
kFPGain               <-  -2000

#Establezco semilla aleatoria
set.seed(1)



In [4]:
#cargo los datos
dataset <- fread(karchivo_entrada)

#Para hacer pruebas rapidas puedo reducir el dataset a una fraccion
#subsample <- sample(1:nrow(dataset), .1 * nrow(dataset))
#dataset <- dataset[subsample,]

dtrain  <- dataset[ foto_mes==202101 ]  #defino donde voy a entrenar

train_rows <- createDataPartition(dtrain$clase, p = .66, list = FALSE)

#train_rows <- sample(1:nrow(dataset), .66 * nrow(dataset))
dtest <- dtrain[-train_rows,]
dtrain <- dtrain[train_rows,]

dapply  <- dataset[ foto_mes==202103 ]  #defino donde voy a aplicar el modelo

In [8]:
#Funcion que calcula la ganancia para una prediccion y valores reales
fmetrica_ganancia_rpart  = function( probs, clases, pclase_valor_positivo )
{
 
  return(  sum(    (probs > 1/40 ) * 
                   ifelse( clases== pclase_valor_positivo, kTPGain, kFPGain )   
              )
         )
}

In [6]:
# Funcion que entrena el arbol y luego determina la ganancia sobre el set de test
modelo_rpart_ganancia = function( ptrain, ptest, pmaxdepth, pminbucket, pminsplit, pcp )
{

  modelo   <-  rpart("clase_ternaria ~ .",   data = ptrain,  xval=0, maxdepth=pmaxdepth, minbucket=pminbucket, minsplit=pminsplit, cp=pcp)

  #aplico el modelo a datos nuevos
  testing_prediccion  <- predict( modelo, ptest)[, "BAJA+2"]
    
  return(fmetrica_ganancia_rpart(testing_prediccion,ptest[,..kclase_nomcampo],"BAJA+2"))
  
}

In [9]:
#Cuenta los modelos probados
linea <- 1

#Establece una ganancia inicial muy baja a superar
max_gan <- -100000000

#For anidados para grid search 
for( vcp in c( 0, 0.01, 0.001)) #Complexity Factor
for( vmaxdepth in  c(4,8,14,16) ) #max Depth
for( vminsplit in  c(2, 4, 8, 10, 15, 20, 30, 50, 100, 150, 200, 300, 400 ) ) #Min Split
for( vminbucket  in  unique( as.integer(c(1,2,3,4,5,  vminsplit/10, vminsplit/5, vminsplit/3, vminsplit/2 )) ) ) #Min Bucket
{ 

    #Muestra los parametros a testear
    message(paste('Intentando',vcp,vminsplit,vminbucket,vmaxdepth))
    #Llama a la funcion que entrena el modelo y calcula la ganancia para esos parametros
    res <- modelo_rpart_ganancia( dtrain, dtest, 
                                  pmaxdepth=vmaxdepth, pminbucket=vminbucket, pminsplit=vminsplit, pcp=vcp )

  
    #Verifica si se supero la mejor ganancia hasta el momento. En caso de haberla superado graba los hiperparametros nuevos
    if (res>max_gan) {
      max_gan <- res
      message(paste("Nuevo Maximo: ", res*3, vcp,vminsplit,vminbucket,vmaxdepth))
      pcp<-vcp
      pminsplit<-vminsplit
      pminbucket<-vminbucket
      pmaxdepth<-vmaxdepth
  }
  #Muestra al ganancia de la iteracion
  message(paste('Ganancia:',res*3,'Ganancia Maxima',max_gan*3))
    
  linea <- linea+1


}

Intentando 0 2 1 4

Nuevo Maximo:  17220000 0 2 1 4

Ganancia: 17220000 Ganancia Maxima 17220000

Intentando 0 2 2 4

Ganancia: 0 Ganancia Maxima 17220000

Intentando 0 2 3 4

Ganancia: 0 Ganancia Maxima 17220000

Intentando 0 2 4 4

Ganancia: 0 Ganancia Maxima 17220000

Intentando 0 2 5 4

Ganancia: 0 Ganancia Maxima 17220000

Intentando 0 2 0 4

Ganancia: 17220000 Ganancia Maxima 17220000

Intentando 0 4 1 4

Ganancia: 17220000 Ganancia Maxima 17220000

Intentando 0 4 2 4

Ganancia: 0 Ganancia Maxima 17220000

Intentando 0 4 3 4

Ganancia: 0 Ganancia Maxima 17220000

Intentando 0 4 4 4

Ganancia: 0 Ganancia Maxima 17220000

Intentando 0 4 5 4

Ganancia: 0 Ganancia Maxima 17220000

Intentando 0 4 0 4

Ganancia: 17220000 Ganancia Maxima 17220000

Intentando 0 8 1 4

Ganancia: 17220000 Ganancia Maxima 17220000

Intentando 0 8 2 4

Ganancia: 0 Ganancia Maxima 17220000

Intentando 0 8 3 4

Ganancia: 0 Ganancia Maxima 17220000

Intentando 0 8 4 4

Ganancia: 0 Ganancia Maxima 17220000

Inte

In [12]:
#Entrena el modelo con el dataset completo
t0       <-  Sys.time()
modelo   <-  rpart(formula_obj,   data = dataset,   xval=0, maxdepth=pmaxdepth, minbucket=pminbucket, minsplit=pminsplit, cp=pcp)
t1       <-  Sys.time()

tcorrida <-  as.numeric( t1 - t0, units = "secs")
print( tcorrida)

[1] 4.870981


In [13]:
#Detalles del modelo
modelo

n= 238608 

node), split, n, deviance, yval
      * denotes terminal node

 1) root 238608 971.016000 0.004086200  
   2) ctrx_quarter>=18.5 220827 467.008200 0.002119306  
     4) ctrx_quarter>=49.5 185242 186.811200 0.001009490  
       8) ccomisiones_mantenimiento< 23 185241 185.813200 0.001004097 *
       9) ccomisiones_mantenimiento>=23 1   0.000000 1.000000000 *
     5) ctrx_quarter< 49.5 35585 278.781100 0.007896586  
      10) Master_mpagominimo< 706832.1 35584 277.796800 0.007868705 *
      11) Master_mpagominimo>=706832.1 1   0.000000 1.000000000 *
   3) ctrx_quarter< 18.5 17781 492.543600 0.028513580  
     6) mpasivos_margen>=34.305 8946 132.962800 0.015090540 *
     7) mpasivos_margen< 34.305 8835 356.336800 0.042105260  
      14) mprestamos_personales>=36166.5 1212   9.917492 0.008250825  
        28) mcuenta_corriente< 1586.375 1211   8.933113 0.007431874 *
        29) mcuenta_corriente>=1586.375 1   0.000000 1.000000000 *
      15) mprestamos_personales< 36166.5 7623 3

In [14]:
#Leo datos a estimar
dataset_pred <- fread(karchivo_predecir)


#Genero la clase
#dataset_pred[,(kobjetivo) := as.integer(dataset[,..kclase_nomcampo] == kclase_valor_positivo)]

#Limpio campos a borrar
#dataset_pred[ ,  (kcampos_a_borrar) := NULL    ] 

prediccion  <- predict( modelo, dataset_pred ) #aplico el modelo

In [15]:
dataset_pred[ , prob_baja := prediccion]
dataset_pred[ , Predicted  := as.numeric(prob_baja > 0.025) ]

entrega  <- dataset_pred[   , list(numero_de_cliente, Predicted) ] #genero la salida

#genero el archivo para Kaggle
fwrite( entrega, file="./salida_rpart_grid.csv", sep="," )